In [1]:
from amogel.model.GCN import GCN
import torch
from torch_geometric.utils import to_dense_batch
from torch_geometric.loader import DataLoader
from captum.attr import IntegratedGradients
from tqdm import tqdm
from amogel.utils.common import load_omic_features_name 

/home/student/anaconda3/envs/gnn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test_graph = torch.load("../artifacts/amogel/test_graph.pt")
test_loader = DataLoader(test_graph, batch_size=50, shuffle=False)
model = GCN(3, 32 , 3 , test_graph[0].x.shape[0] * test_graph[0].x.shape[1])
model.load_state_dict(torch.load("../artifacts/amogel/BRCA_model.pt"))

class NewModel(torch.nn.Module):
    def __init__(self, model):
        super(NewModel, self).__init__()
        self.A = model.feedforward 
        self.B = model.mlp
        self.z = torch.zeros(50, 64) # batch_size
        
    def forward(self, x , gnn_x):
        i = self.A(x)
        #print(i.shape)
        #print(self.z.shape)
        y  = torch.concat([gnn_x , i], dim=-1)
        #print(y.shape)
        return self.B(y)
 
new_model = NewModel(model)
new_model.eval()

actual = []
attribution = []
with tqdm(total=len(test_loader)) as pbar:
    for batch in test_loader: 
        output2 , _ , _ , _ , _ , _ , gnn_x = model(batch.x , batch.edge_index , batch.edge_attr , batch.batch)
        x , edge_index , edge_attr, batch , y = batch.x , batch.edge_index , batch.edge_attr , batch.batch , batch.y
        batch_x , batch_mask = to_dense_batch(x , batch) # dimension ( no_batch , number_of_node )
        batch_x = batch_x.view(batch_x.shape[0] , batch_x.shape[1] * batch_x.shape[2]) 
        #print(batch_x.shape)
        #print(y)
        #print(new_model(batch_x).shape)
        output = new_model(batch_x , gnn_x)
        # calculate accuracy
        #print(output.argmax(dim=-1))
        #print((output.argmax(dim=1) == y).sum().item() / y.shape[0])
        #print(output2.argmax(dim=-1))
        #print((output2.argmax(dim=1) == y).sum().item() / y.shape[0])
        
        ig = IntegratedGradients(new_model)
        attr , app_err = ig.attribute((batch_x , gnn_x) , (torch.zeros_like(batch_x) , gnn_x) , return_convergence_delta=True , target=y)
        #print(attr[0][0][0:9])
        #print(attr[0].reshape(50, -1 , 3).sum(dim=-1))
        actual.append(y)
        attribution.append(attr[0].reshape(batch_x.shape[0], -1 , 3).sum(dim=-1))
        pbar.update(1)

#print(test_graph[0].x.shape[0] * test_graph[0].x.shape[1])

RuntimeError: Error(s) in loading state_dict for GCN:
	size mismatch for lin.weight: copying a param with shape torch.Size([5, 32]) from checkpoint, the shape in current model is torch.Size([3, 32]).
	size mismatch for lin.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([3]).
	size mismatch for mlp.8.weight: copying a param with shape torch.Size([5, 32]) from checkpoint, the shape in current model is torch.Size([3, 32]).
	size mismatch for mlp.8.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([3]).

In [25]:
total_actual = torch.cat(actual , dim=0)
total_attr = torch.cat(attribution , dim=0)

In [78]:
import seaborn as sns
# heatmap if actual = 0
heatmap = []
for idx, output in enumerate(total_attr):
    actual = total_actual[idx]
    if actual == 2:
        #mask = (output <= 0.01)
        mask_output = output.clone()
        #mask_output[mask] = 0
        heatmap.append(mask_output)
        
#sns.heatmap(heatmap)

omic_types = [ 1 , 2 , 3 ]
feature_names = load_omic_features_name(
    "../artifacts/data_preprocessing/" , "KIPAN" , [1,2,3]
)

# ac selected genes 
ac_genes = torch.load("../artifacts/ac_genes/gene.pt" , map_location=torch.device("cpu"))
filter_df = feature_names[feature_names["gene_idx"].isin(ac_genes)]

indices = torch.stack(heatmap, dim=0).sum(dim=0).topk(20).indices
for i in indices:
    print(filter_df.iloc[int(i)].gene_name)

mir1271
SPRY1
PART1
CDH13
CAV1
mir26a1
C2orf62
FAM184B
CHKB
mir424
UBE2N
mir561
MGAT4B
DTNA
SPATA12
KCP
FMNL3
C22orf45
BARX2
GPR124


In [ ]:
import torch 
from captum.attr import IntegratedGradients

In [ ]:
model.feedforward
model.mlp

class NewModel(torch.nn.Module):
    def __init__(self, model):
        super(NewModel, self).__init__()
        self.A = model.feedforward 
        self.B = model.mlp
        self.z = torch.zeros(10, 64)
        
    def forward(self, x):
        i = self.A(x)
        print(i.shape)
        print(self.z.shape)
        y  = torch.concat([i , torch.zeros_like(i)], dim=-1)
        print(y.shape)
        return self.B(y)
 
new_model = NewModel(model)

new_model.eval()
        

In [ ]:
new_model(torch.rand(10, 100))

In [ ]:
A = torch.randn(10 , 100)
B = torch.zeros(10 , 100)

ig = IntegratedGradients(new_model)
attr , app_err = ig.attribute(A , B , return_convergence_delta=True , target=[0,1,2,3,4,0,1,2,3,4])
attr.shape

In [ ]:
attr[0]